In [1]:
import torch
from transformers import BartForConditionalGeneration
import pandas as pd 
from datasets import *

from sklearn.model_selection import train_test_split

/Users/kayhan/opt/anaconda3/envs/pendulum/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-09 02:57:05.580030: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Experiment notes


- BART encoder input size is too short 
- 1 char is approx 4 tokens
- our input and output sizes are 2500 and 150 tokens


accidentally trained the model without stripping the original part. 

In [7]:
path = '/Users/kayhan/Downloads/data_to_label.csv'
# df = pd.read_csv('../labeling jobs/data_to_label.csv')
df = pd.read_csv(path)
data = df[['name','review','summary']]
data['review'] = data['review'].apply(lambda x : x.replace(rmv_str,''))

/var/folders/vy/y5363_9d6f54v8kgq1spp7n80000gn/T/ipykernel_7837/2969053172.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review'] = data['review'].apply(lambda x : x.replace(rmv_str,''))


In [8]:
data_train, data_test = train_test_split(data,test_size=0.2,random_state=0)

In [9]:
train_dataset = Dataset.from_pandas(data_train,split='train')
test_dataset = Dataset.from_pandas(data_test,split='test')
ds_dict = {'train':train_dataset,'test':test_dataset}
dataset = DatasetDict(ds_dict)
dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'review', 'summary', '__index_level_0__'],
        num_rows: 275
    })
    test: Dataset({
        features: ['name', 'review', 'summary', '__index_level_0__'],
        num_rows: 69
    })
})

In [10]:
dataset['test'][0]

{'name': 'Hu Tong Jian Bing',
 'review': " Love that this is an addition to the Dogpatch area! I remember when this first came to SF I waited 30 min in line at a SPARK food festival to get my hands on one. So nice to walk in and order in an actual brick and mortar joint! This isn't like the traditional ones I've had in China but they're still tasty! The only downsides are as others have mentioned:1. The sauce isn't as spicy as flavorful as I had hoped. Even with the addition of the $1 picked veggie it still lacked a kick. I'm going to order extra spicy with extra sauce next time.2. The outside was a bit undercooked and doughy. Still good but it wasn't as well done as it could be. Still going to frequent this joint though.,I got the roast duck jian bing and the mango sago. Both were really good and reminded me of authentic jian bing!,Happy Cow Creamery & Tea is an Asian dessert spot in the Dogpatch. I've recently just noticed it and was surprised to find out that they house a permanent 

In [11]:
import evaluate

rouge = evaluate.load("rouge")

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from transformers import LEDForConditionalGeneration, LEDTokenizer


# checkpoint = 'allenai/led-base-16384'
# checkpoint = 'allenai/PRIMERA'
checkpoint = 'facebook/bart-base'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
# tokenizer = LEDTokenizer.from_pretrained(checkpoint)
# model = LEDForConditionalGeneration.from_pretrained(checkpoint)
# model = BartForConditionalGeneration.from_pretrained(checkpoint)


# from transformers import (

#     AutoTokenizer,

#     LEDConfig,

#     LEDForConditionalGeneration,

# )

# tokenizer = AutoTokenizer.from_pretrained('allenai/PRIMERA')

# config=LEDConfig.from_pretrained('allenai/PRIMERA')

# model = LEDForConditionalGeneration.from_pretrained('allenai/PRIMERA')


In [31]:
df['name'].unique()

array(['Alnico', 'Arang', 'Automat', 'Beit Rima', 'Beretta', 'Bottega',
       'Breadbelly', 'Cafe De Casa', 'CreoLa',
       'Daeho Kalbijjim & Beef Soup', 'Dumpling Home', 'Dumpling House',
       'Dumpling Kitchen', 'Fable', 'Farmhouse Kitchen Thai Cuisine',
       'Fog Harbor Fish House', 'Gao Viet Kitchen & Bar', 'Horsefeather',
       'Hu Tong Jian Bing', 'JIJIME', "Jamie's Place", 'KAIYŌ Rooftop',
       'Kothai Republic', 'Lily', 'Limoncello', 'Lokma', 'Marufuku Ramen',
       'Noodle in a Haystack', 'Nopa', 'Otra', 'Pearl', 'Pinsa Rossa',
       'Purple Rice', 'Rise Over Run', 'Routier', 'San Tung', 'Santeria',
       'Savor', 'Sotto Mare', 'Starbelly', 'Sweet Maple', 'Tanglad',
       "That's My Jam", 'The Italian Homemade Company', 'The Laundromat',
       'The Snug', "The Tailor's Son", 'Um Ma', 'Wooden Spoon'],
      dtype=object)

In [42]:
example_text = dataset['train']
example_text = example_text.filter(lambda x : x['name'] == 'Marufuku Ramen')[0]

100%|██████████| 1/1 [00:00<00:00, 179.05ba/s]


In [43]:
# len(example_text)

In [44]:
encoded_text = tokenizer.encode(example_text['review'],return_tensors='pt',max_length=1024)


In [45]:
encoded_text

tensor([[   0, 8432,   10,  ...,  828,    9,    2]])

In [46]:
output = model.generate(encoded_text,max_length=150)

In [47]:
print(tokenizer.decode(output[0]))

</s><s> Got a table for one and was on the waitlist for a really long time. It took about an hour and a half to get a text to see the hostess. The only tiny thing that bugged me a bit is that they kept trying to rush me out since I was a tablefor one and every time I took a break from eating to get my breather or to just peek at my phone, they kept asking if I was done and needed a to-go box. Aside from that, the food is really good and worth the wait and everyone that works there is super attentive and checks in on you and your cup is never empty!! The Hakata Tonkotsu was so flavorful and really hits the spot on</s>


Got a table for one and was on the waitlist for a really long time. It took about an hour and a
 half to get a text to see the hostess. The only tiny thing that bugged me a bit is that they kept 
trying to rush me out since I was a tablefor one and every time I took a break from eating to get my breather
or to just peek at my phone, they kept asking if I was done and needed a to-go box. Aside from that, the food 
is really good and worth the wait and everyone that works there is super attentive and checks in on you and your
cup is never empty!! The Hakata Tonkotsu was so flavorful and really hits the spot on

summarize:  I tried the strawberry balsamic danish and it was amazing! It was so fresh and tasty, the pastry was so fluffy and buttery as well.

 The ube tart was also delicious and it wasn't too sweet. I also got the 50/50 iced and it also really good! There was a good balance between the coffee
 
  taste and sweetness of the drink.,Like all the other reviews below - the Kaya toast is so good! I decided to get it with the Gyeranbbang which was recommended
  
   by the staff there. They paired so well because one was sweet and the other savory!,I think my bestie outdid himself and took me to all the best pastry shops

In [31]:
tokenizer.model_max_length

1024

In [32]:
df['review'].apply(len).median() / 4

2348.25

In [33]:
df['summary'].apply(len).median() / 4

128.5

In [34]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["review"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=150, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [35]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12.06ba/s]


In [36]:
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [37]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [38]:
training_args = Seq2SeqTrainingArguments(
    output_dir="generative_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: review, summary, name, __index_level_0__. If review, summary, name, __index_level_0__ are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/Users/kayhan/opt/anaconda3/envs/pendulum/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 275
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 72
  Number of trainable parameters = 139420416
You're using a BartTokenizerFast tokenizer

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.194865,0.198300,0.092500,0.168600,0.168900,20.000000
2,No log,2.108056,0.200600,0.098100,0.172000,0.172100,20.000000
3,No log,2.087942,0.203800,0.095300,0.170800,0.170900,20.000000
4,No log,2.078477,0.202100,0.097600,0.170400,0.170500,20.000000


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: review, summary, name, __index_level_0__. If review, summary, name, __index_level_0__ are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 69
  Batch size = 16
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate co

TrainOutput(global_step=72, training_loss=2.5621821085611978, metrics={'train_runtime': 5177.4771, 'train_samples_per_second': 0.212, 'train_steps_per_second': 0.014, 'total_flos': 670710104064000.0, 'train_loss': 2.5621821085611978, 'epoch': 4.0})

In [56]:
eval_metrics = trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: review, summary, name, __index_level_0__. If review, summary, name, __index_level_0__ are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 69
  Batch size = 16
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,

In [57]:
eval_metrics

{'eval_loss': 2.078477382659912,
 'eval_rouge1': 0.2021,
 'eval_rouge2': 0.0976,
 'eval_rougeL': 0.1704,
 'eval_rougeLsum': 0.1705,
 'eval_gen_len': 20.0,
 'eval_runtime': 179.0687,
 'eval_samples_per_second': 0.385,
 'eval_steps_per_second': 0.028,
 'epoch': 4.0}

In [39]:
tuned_model = trainer.model

In [59]:
input_ = dataset['test']['review'][-2]

In [60]:
encoded = tokenizer.encode(input_,return_tensors='pt',max_length=1024)

,0
eval_loss,2.078477
eval_rouge1,0.202100
eval_rouge2,0.097600
eval_rougeL,0.170400
eval_rougeLsum,0.170500
eval_gen_len,20.000000
eval_runtime,179.068700
eval_samples_per_second,0.385000
eval_steps_per_second,0.028000
epoch,4.000000


In [62]:
output = tuned_model.generate(input_ids=encoded,max_length=150)

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



In [63]:
tokenizer.decode(output[0])

'</s><s>Dungeness crab pave, spiced mayonnaise - Dungeness Crab pave is a delicious dessert with a California flair. The restaurant is located on the 2nd floor with tables and a bar area. However, the waiters are not organized and disorganized. The food is mediocre, but the service is excellent. The staff is friendly and attentive, and the atmosphere is convivial. Overall, the restaurant is a must-visit.</s>'

In [64]:
 dataset['test']['summary'][-2]

'Routier'

In [53]:
 dataset['test']['review'][-1]

" Good breakfast place. Tried Millionaire bacon which was super tasty! South of the border was really good and the potato was crispy and yummy tasting. The pancakes were super good! The OJ was fresh but not the best probably the only thing we didn't like as much. Service was fast staff was very attentive. Parking wasn't too basic we found one right away. Overall it was a good restaurant would definitely reccomend.,i've always wanted to come here but there's alwayyyys a long line!! but i got very lucky coming here on a random thursday after a doctors appointment :) i got the pimped shin cup bop and it was so good. It's also a great amount of food so i took it to go and my leftovers were just as delish. so happy i got to come here. the only thing i wasn't a fan of was the candy bacon we got-- but everything else outweighed it. will definitely come back!,Absolutely great brunch spot in San Francisco! I've had this spot several times, and it consistently delivers each time. They usually ha

In [57]:
tokenizer.decode(output[0])

'</s><s>Alnico is a Vietnamese restaurant in San Francisco that serves delicious Vietnamese coffee with Ube Foam. The food and drinks are delicious, but the price is reasonable for everything that comes with the ube flavored stuff. The restaurant is located on Hill St. and has outdoor seating available and doggo friendly seating. The service is good and the food is good. However, the price can be high for a weekend brunch.</s>'

In [63]:
 dataset['test']['review'][2]

'use the following opinions of the restaurant reviews to create a 50 word summary: \n The very definition of a hidden gem.I found this place on Yelp and my friend made a reservation for 6 people (they DID require a "deposit")This place has limited indoor seating due to the super cool 2 story layout. Kinda a small dining room on main level, then a balcony of seats on floor 2 that surrounds the dining room on 2 sides. ¬†Our table of 6 was all by itself on the right upper level- so perfect for us to all catch up with each other.Our FANTASTIC waiter Frederick was friendly and attentive, I wanted a lot of ice, he brought us 2 pitchers of ice! We ordered drinks (note that due to them only have a wine/beer license, their "cocktails" are made with low-proof rum or vodka.)We ordered some fried okra and some corn muffins to start and share. ¬†The okra comes with a snappy dipping sauce. (Note the corn muffins are pretty small).I ordered the fried chicken (leg and thigh) with Jambalaya and the col

In [25]:
test = tokenized_dataset['test']['input_ids'][0]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('allenai/PRIMERA')
tokenizer.model_max_length

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('allenai/led-large-16384')
tokenizer.model_max_length

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
tokenizer.model_max_length

In [ ]:

tokenizer = AutoTokenizer.from_pretrained('google/pegasus-large')
# model = GPT2Model.from_pretrained('gpt2')
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)
tokenizer.model_max_length